In [1]:
import jax
import jax.numpy as jnp
from jax import jit, vmap, grad
from jax import random
import optax
import numpy as np
import pickle

In [2]:
def init_network_params(sizes, key=random.PRNGKey(4)):
    def random_layer_params(m, n, key, scale=1e-2):
        w_key, b_key = random.split(key)
        return scale * random.uniform(w_key, (n, m), minval=1, maxval=3), scale * random.uniform(b_key, (n,), minval=1, maxval=3)

    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

def predict(NN, x):
    activations = x.reshape((-1, ))
    for w, b in NN[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = jax.nn.sigmoid(outputs)
    final_w, final_b = NN[-1]
    logit = jnp.dot(final_w, activations) + final_b
    logit = 1/(1+jnp.exp(-logit))
    return logit.squeeze()

vpredict = vmap(predict, in_axes=(None, 0))
gpredict = grad(predict, argnums=1)
vgpredict = vmap(gpredict, in_axes=(None, 0))

In [3]:
def pi(x):
    return (1 / jnp.sqrt(2 * jnp.pi)) * jnp.exp(-0.5 * x**2)
vpi = vmap(pi, in_axes=(0,))

def pi_pre(NN, x):
    result = gpredict(NN, x)
    return result

@jit
def obj(NN, x_batch):
    def obj1(NN, x):
        return pi(x) - pi_pre(NN, x)
    vobj1 = vmap(obj1, in_axes=(None, 0))

    result = vobj1(NN, x_batch)
    result = jnp.square(result)
    result = jnp.mean(result)
    return result
jgo = jit(grad(obj))

@jit
def evaluation(NN, test_zu):
    z_batch, u_batch = test_zu[:, 0], test_zu[:, 1]
    result = jnp.square(vpredict(NN, z_batch) - u_batch)
    result = jnp.mean(result)
    return result

In [ ]:
LAYER_SIZES = [1, 10, 10, 10, 1]
NN = init_network_params(LAYER_SIZES, random.PRNGKey(15))

optimizer= optax.adam(0.001)
opt_state = optimizer.init(NN)
test_zu = np.load('/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/test_zu.npy')
lowest = evaluation(NN, test_zu)
Lloss = []
Llowest = []
for epoch in range(2000000):
    x_batch = np.random.uniform(-2, 2, (10000, 1)) #1000 refers to batch size

    grads = grad(obj)(NN, x_batch)
    updates, opt_state = optimizer.update(grads, opt_state)
    NN = optax.apply_updates(NN, updates)

    Lloss.append(obj(NN, x_batch).item())
    Llowest.append(lowest.item())
    print(f"Epoch: {epoch}, loss:{obj(NN, x_batch)}, {lowest}")

    if  evaluation(NN, test_zu)<lowest:
        lowest = evaluation(NN, test_zu)
        NN_best = NN

    if epoch % 10000 == 0:
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/Interval 2/Lloss.npy', Lloss)
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/Interval 2/Llowest.npy', Llowest)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/Interval 2/NN (best).pkl', 'wb') as f:
            pickle.dump(NN_best, f)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (PINN)/Interval 2/NN_{epoch}.pkl', 'wb') as f:
            pickle.dump(NN, f)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch: 1592652, loss:1.6178866024674221e-09, 1.8419171965433634e-06
Epoch: 1592653, loss:1.5702004141360248e-09, 1.8419171965433634e-06
Epoch: 1592654, loss:1.52665358132964e-09, 1.8419171965433634e-06
Epoch: 1592655, loss:1.3724672509596303e-09, 1.8419171965433634e-06
Epoch: 1592656, loss:1.3128275133666989e-09, 1.8419171965433634e-06
Epoch: 1592657, loss:1.3218701688799683e-09, 1.8419171965433634e-06
Epoch: 1592658, loss:1.5150189991430807e-09, 1.8419171965433634e-06
Epoch: 1592659, loss:1.6805861147162204e-09, 1.8419171965433634e-06
Epoch: 1592660, loss:1.7618517755835228e-09, 1.8419171965433634e-06
Epoch: 1592661, loss:1.6851859907518474e-09, 1.8419171965433634e-06
Epoch: 1592662, loss:1.645848124454119e-09, 1.8419171965433634e-06
Epoch: 1592663, loss:1.6102384980953843e-09, 1.8419171965433634e-06
Epoch: 1592664, loss:1.4011293236748656e-09, 1.8419171965433634e-06
Epoch: 1592665, loss:1.397679083581238e-09, 1.8419171965433634e-06
Epoch: 1592666, loss:1.37